In [12]:
import os
os.getcwd()

'D:\\Documents\\PycharmProjects\\LEADING-RAMP'

Script from Jon (modify file path to datasets) 

In [13]:
import pandas as pd
from zipfile import ZipFile
import glob

In [14]:
def extract_subset_ramp_data(zip_file, ir_repo_id):
    """This function tries to conserve memory by opening zipped RAMP monthly
       data files one at a time and subsetting the data to a single repository's
       data for that month, prior to further processing or aggregation.

    Parameters
    ----------

    zip_file:
        String. A file path pointing to a zip file.

    ir_repo_id:
        String. A locally unique repository identifier which will be used to
        subset the unzipped data.

    Returns
    -------

    ir_data:
        A Pandas dataframe. The subset of RAMP data for the specified
        repository and month.

    """
    with ZipFile(zip_file) as rampzip:
        with rampzip.open(rampzip.namelist()[0]) as rampfile:
            ramp_df = pd.read_csv(rampfile)
    ir_data = ramp_df[ramp_df["repository_id"] == ir_repo_id].copy()
    return ir_data


In [15]:
def get_ir_data(ir_repo_id, cols, file_list):
    """This function iterates through a list of zipped RAMP data files to
       aggregate a subset of complete RAMP data for a single IR. Creates
       an empty Pandas dataframe and then appends monthly data to it.

    Parameters
    ----------

    ir_repo_id:
        String. A locally unique repository identifier, designating the
        repository whose data will be aggregated.

    cols:
        List. Column names to be used in the empty dataframe. Appended data will
        have the same columns by default.

    file_list:
        List. File paths to monthly RAMP data in zipped format.

    Returns
    -------

    ir_data:
        A Pandas dataframe. The aggregated RAMP data for the specified IR
        across all months included in the file_list.

    """
    ir_data = pd.DataFrame(columns=cols)
    # append data from each month
    for mo_data in file_list:
        print(mo_data)
        ir_data = ir_data.append(extract_subset_ramp_data(mo_data, ir_repo_id))
    return ir_data

In [16]:
def get_v1_data(ir_repo_id):
    """This function aggregates all RAMP data that was harvested for a single IR
       between January 1, 2017 and August 18, 2018 ("v1" data). Column names
       for those data are hard coded. A list of data files is also generated.

       TODO: It may be useful to remove this function and subset the v1
       data for each month as part of the v2 aggregations. Leaving it for
       now in case we want the option to store/save v1 data by itself.

    Parameters
    ----------

    ir_repo_id:
        String. A locally unique repository identifier, designating the
        repository whose data will be aggregated.

    Returns
    -------

    ir_v1_data:
        A Pandas dataframe. The aggregated RAMP "v1" data for the specified IR
        across all months included in the all_data_file_list.
    """
    all_cols = ['citableContent', 'clickThrough', 'clicks', 'country', 'date',
                'device', 'impressions', 'index', 'position', 'url',
                'repository_id']    
    
    all_data_file_list = glob.glob("D:/Documents/PycharmProjects/LEADING-RAMP/ir_data/Ramp2019/*all.zip")
    ir_v1_data = get_ir_data(ir_repo_id, all_cols, all_data_file_list)
    return ir_v1_data


In [17]:
def get_v2_pc_data(ir_repo_id):
    """This function aggregates all RAMP page click data harvested for a single
       IR since August 19, 2018 ("v2" data). Column names for those data are
       hard coded. A list of data files is also generated.

    Parameters
    ----------

    ir_repo_id:
        String. A locally unique repository identifier, designating the
        repository whose data will be aggregated.

    Returns
    -------

    ir_v2_pc_data:
        A Pandas dataframe. The aggregated RAMP page clicks data for the
        specified IR across all months included in the pageclick_data_file_list.
    """
    pageclick_cols = ['citableContent', 'clickThrough', 'clicks', 'date',
                      'impressions', 'index', 'position', 'url',
                      'repository_id']
    pageclick_data_file_list = glob.glob("D:/Documents/PycharmProjects/LEADING-RAMP/ir_data/Ramp2019/*all_page-clicks.zip")
    ir_v2_pc_data = get_ir_data(ir_repo_id, pageclick_cols,
                                pageclick_data_file_list)
    return ir_v2_pc_data

In [18]:
def concat_ramp_versions(v1_data, v2_data):
    subset_cols = list(v2_data.columns.values)
    v1_subset = v1_data[subset_cols].copy()
    v1_v2_concatenated = pd.concat([v1_subset, v2_data], ignore_index=True)
    return v1_v2_concatenated

In [19]:
def process_repo(ir_repo_id):
    """This is basically a workflow function that calls all the other functions.

    Parameters
    ----------

    ir_repo_id:
       String. A locally unique identifier for the repository whose data will
       be aggregated.

    Returns
    -------

    ir_complete_pc_data:
       A Pandas dataframe. The aggregated page click data across all the
       years/months specified for the specified repository.

    ir_complete_ai_data:
       A Pandas dataframe. The aggreated country/device data across all the
       years/months specified for the specified repository.

    """
    ir_v1_data = get_v1_data(ir_repo_id)
    ir_v2_pc_data = get_v2_pc_data(ir_repo_id)
    #ir_v2_ai_data = get_v2_ai_data(ir_repo_id)
    ir_complete_pc_data = concat_ramp_versions(ir_v1_data, ir_v2_pc_data)
    #ir_complete_ai_data = concat_ramp_versions(ir_v1_data, ir_v2_ai_data)
    return ir_complete_pc_data #, ir_complete_ai_data

In [20]:
msu_pc_2019 = process_repo("montana_state_university")

D:/Documents/PycharmProjects/LEADING-RAMP/ir_data/Ramp2019\2019-01_RAMP_all_page-clicks.zip
D:/Documents/PycharmProjects/LEADING-RAMP/ir_data/Ramp2019\2019-02_RAMP_all_page-clicks.zip
D:/Documents/PycharmProjects/LEADING-RAMP/ir_data/Ramp2019\2019-03_RAMP_all_page-clicks.zip
D:/Documents/PycharmProjects/LEADING-RAMP/ir_data/Ramp2019\2019-04_RAMP_all_page-clicks.zip
D:/Documents/PycharmProjects/LEADING-RAMP/ir_data/Ramp2019\2019-05_RAMP_all_page-clicks.zip
D:/Documents/PycharmProjects/LEADING-RAMP/ir_data/Ramp2019\2019-06_RAMP_all_page-clicks.zip
D:/Documents/PycharmProjects/LEADING-RAMP/ir_data/Ramp2019\2019-07_RAMP_all_page-clicks.zip
D:/Documents/PycharmProjects/LEADING-RAMP/ir_data/Ramp2019\2019-08_RAMP_all_page-clicks.zip
D:/Documents/PycharmProjects/LEADING-RAMP/ir_data/Ramp2019\2019-09_RAMP_all_page-clicks.zip
D:/Documents/PycharmProjects/LEADING-RAMP/ir_data/Ramp2019\2019-10_RAMP_all_page-clicks.zip
D:/Documents/PycharmProjects/LEADING-RAMP/ir_data/Ramp2019\2019-11_RAMP_all_page

In [21]:
msu_pc_2019 

,citableContent,clickThrough,clicks,date,impressions,index,position,url,repository_id
0,No,0.000000,0.0,2019-01-04,1.0,montana_page_clicks,51.000000,https://scholarworks.montana.edu/xmlui/handle/...,montana_state_university
1,Yes,0.000000,0.0,2019-01-06,4.0,montana_page_clicks,63.000000,https://scholarworks.montana.edu/xmlui/bitstre...,montana_state_university
2,Yes,0.000000,0.0,2019-01-08,36.0,montana_page_clicks,87.777778,https://scholarworks.montana.edu/xmlui/bitstre...,montana_state_university
3,Yes,0.105263,2.0,2019-01-09,19.0,montana_page_clicks,10.894737,https://scholarworks.montana.edu/xmlui/bitstre...,montana_state_university
4,Yes,0.000000,0.0,2019-01-06,2.0,montana_page_clicks,24.000000,https://scholarworks.montana.edu/xmlui/bitstre...,montana_state_university
...,...,...,...,...,...,...,...,...,...
1707663,No,0.000000,0.0,2019-12-25,9.0,montana_page_clicks,61.111111,https://scholarworks.montana.edu/xmlui/handle/...,montana_state_university
1707664,No,0.000000,0.0,2019-12-25,1.0,montana_page_clicks,68.000000,https://scholarworks.montana.edu/xmlui/handle/...,montana_state_university
1707665,No,0.000000,0.0,2019-12-25,1.0,montana_page_clicks,93.000000,https://scholarworks.montana.edu/xmlui/handle/...,montana_state_university
1707666,No,0.000000,0.0,2019-12-25,1.0,montana_page_clicks,39.000000,https://scholarworks.montana.edu/xmlui/recent-...,montana_state_university


In [22]:
msu_pc_2019.to_csv("D://Documents//PycharmProjects//LEADING-RAMP//ir_data//Ramp2019//Export/msu_pc_2019.csv", header = True)

In [23]:
boston_pc_2019 = process_repo("boston_university")

D:/Documents/PycharmProjects/LEADING-RAMP/ir_data/Ramp2019\2019-01_RAMP_all_page-clicks.zip
D:/Documents/PycharmProjects/LEADING-RAMP/ir_data/Ramp2019\2019-02_RAMP_all_page-clicks.zip
D:/Documents/PycharmProjects/LEADING-RAMP/ir_data/Ramp2019\2019-03_RAMP_all_page-clicks.zip
D:/Documents/PycharmProjects/LEADING-RAMP/ir_data/Ramp2019\2019-04_RAMP_all_page-clicks.zip
D:/Documents/PycharmProjects/LEADING-RAMP/ir_data/Ramp2019\2019-05_RAMP_all_page-clicks.zip
D:/Documents/PycharmProjects/LEADING-RAMP/ir_data/Ramp2019\2019-06_RAMP_all_page-clicks.zip
D:/Documents/PycharmProjects/LEADING-RAMP/ir_data/Ramp2019\2019-07_RAMP_all_page-clicks.zip
D:/Documents/PycharmProjects/LEADING-RAMP/ir_data/Ramp2019\2019-08_RAMP_all_page-clicks.zip
D:/Documents/PycharmProjects/LEADING-RAMP/ir_data/Ramp2019\2019-09_RAMP_all_page-clicks.zip
D:/Documents/PycharmProjects/LEADING-RAMP/ir_data/Ramp2019\2019-10_RAMP_all_page-clicks.zip
D:/Documents/PycharmProjects/LEADING-RAMP/ir_data/Ramp2019\2019-11_RAMP_all_page

In [24]:
boston_pc_2019.to_csv("D://Documents//PycharmProjects//LEADING-RAMP//ir_data//Ramp2019//Export/boston_pc_2019.csv", header = True)

In [25]:
caltech_a_pc_2019 = process_repo("california_tech_authors")
caltech_t_pc_2019 = process_repo("california_tech_thesis")

D:/Documents/PycharmProjects/LEADING-RAMP/ir_data/Ramp2019\2019-01_RAMP_all_page-clicks.zip
D:/Documents/PycharmProjects/LEADING-RAMP/ir_data/Ramp2019\2019-02_RAMP_all_page-clicks.zip
D:/Documents/PycharmProjects/LEADING-RAMP/ir_data/Ramp2019\2019-03_RAMP_all_page-clicks.zip
D:/Documents/PycharmProjects/LEADING-RAMP/ir_data/Ramp2019\2019-04_RAMP_all_page-clicks.zip
D:/Documents/PycharmProjects/LEADING-RAMP/ir_data/Ramp2019\2019-05_RAMP_all_page-clicks.zip
D:/Documents/PycharmProjects/LEADING-RAMP/ir_data/Ramp2019\2019-06_RAMP_all_page-clicks.zip
D:/Documents/PycharmProjects/LEADING-RAMP/ir_data/Ramp2019\2019-07_RAMP_all_page-clicks.zip
D:/Documents/PycharmProjects/LEADING-RAMP/ir_data/Ramp2019\2019-08_RAMP_all_page-clicks.zip
D:/Documents/PycharmProjects/LEADING-RAMP/ir_data/Ramp2019\2019-09_RAMP_all_page-clicks.zip
D:/Documents/PycharmProjects/LEADING-RAMP/ir_data/Ramp2019\2019-10_RAMP_all_page-clicks.zip
D:/Documents/PycharmProjects/LEADING-RAMP/ir_data/Ramp2019\2019-11_RAMP_all_page

In [26]:
caltech_a_pc_2019.to_csv("D://Documents//PycharmProjects//LEADING-RAMP//ir_data//Ramp2019//Export/caltech_a_pc_2019.csv", header = True)
caltech_t_pc_2019.to_csv("D://Documents//PycharmProjects//LEADING-RAMP//ir_data//Ramp2019//Export/caltech_t_pc_2019.csv", header = True)

In [27]:
pittsburgh_pc_2019 = process_repo("university_pittsburgh")

D:/Documents/PycharmProjects/LEADING-RAMP/ir_data/Ramp2019\2019-01_RAMP_all_page-clicks.zip
D:/Documents/PycharmProjects/LEADING-RAMP/ir_data/Ramp2019\2019-02_RAMP_all_page-clicks.zip
D:/Documents/PycharmProjects/LEADING-RAMP/ir_data/Ramp2019\2019-03_RAMP_all_page-clicks.zip
D:/Documents/PycharmProjects/LEADING-RAMP/ir_data/Ramp2019\2019-04_RAMP_all_page-clicks.zip
D:/Documents/PycharmProjects/LEADING-RAMP/ir_data/Ramp2019\2019-05_RAMP_all_page-clicks.zip
D:/Documents/PycharmProjects/LEADING-RAMP/ir_data/Ramp2019\2019-06_RAMP_all_page-clicks.zip
D:/Documents/PycharmProjects/LEADING-RAMP/ir_data/Ramp2019\2019-07_RAMP_all_page-clicks.zip
D:/Documents/PycharmProjects/LEADING-RAMP/ir_data/Ramp2019\2019-08_RAMP_all_page-clicks.zip
D:/Documents/PycharmProjects/LEADING-RAMP/ir_data/Ramp2019\2019-09_RAMP_all_page-clicks.zip
D:/Documents/PycharmProjects/LEADING-RAMP/ir_data/Ramp2019\2019-10_RAMP_all_page-clicks.zip
D:/Documents/PycharmProjects/LEADING-RAMP/ir_data/Ramp2019\2019-11_RAMP_all_page

In [28]:
pittsburgh_pc_2019.to_csv("D://Documents//PycharmProjects//LEADING-RAMP//ir_data//Ramp2019//Export/pittsburgh_pc_2019.csv", header = True)

In [29]:
vatech_pc_2019 = process_repo("virginia_tech")

D:/Documents/PycharmProjects/LEADING-RAMP/ir_data/Ramp2019\2019-01_RAMP_all_page-clicks.zip
D:/Documents/PycharmProjects/LEADING-RAMP/ir_data/Ramp2019\2019-02_RAMP_all_page-clicks.zip
D:/Documents/PycharmProjects/LEADING-RAMP/ir_data/Ramp2019\2019-03_RAMP_all_page-clicks.zip
D:/Documents/PycharmProjects/LEADING-RAMP/ir_data/Ramp2019\2019-04_RAMP_all_page-clicks.zip
D:/Documents/PycharmProjects/LEADING-RAMP/ir_data/Ramp2019\2019-05_RAMP_all_page-clicks.zip
D:/Documents/PycharmProjects/LEADING-RAMP/ir_data/Ramp2019\2019-06_RAMP_all_page-clicks.zip
D:/Documents/PycharmProjects/LEADING-RAMP/ir_data/Ramp2019\2019-07_RAMP_all_page-clicks.zip
D:/Documents/PycharmProjects/LEADING-RAMP/ir_data/Ramp2019\2019-08_RAMP_all_page-clicks.zip
D:/Documents/PycharmProjects/LEADING-RAMP/ir_data/Ramp2019\2019-09_RAMP_all_page-clicks.zip
D:/Documents/PycharmProjects/LEADING-RAMP/ir_data/Ramp2019\2019-10_RAMP_all_page-clicks.zip
D:/Documents/PycharmProjects/LEADING-RAMP/ir_data/Ramp2019\2019-11_RAMP_all_page

In [30]:
vatech_pc_2019.to_csv("D://Documents//PycharmProjects//LEADING-RAMP//ir_data//Ramp2019//Export/vatech_pc_2019.csv", header = True)

In [31]:
ut_pc_2019 = process_repo("university_texas")

D:/Documents/PycharmProjects/LEADING-RAMP/ir_data/Ramp2019\2019-01_RAMP_all_page-clicks.zip
D:/Documents/PycharmProjects/LEADING-RAMP/ir_data/Ramp2019\2019-02_RAMP_all_page-clicks.zip
D:/Documents/PycharmProjects/LEADING-RAMP/ir_data/Ramp2019\2019-03_RAMP_all_page-clicks.zip
D:/Documents/PycharmProjects/LEADING-RAMP/ir_data/Ramp2019\2019-04_RAMP_all_page-clicks.zip
D:/Documents/PycharmProjects/LEADING-RAMP/ir_data/Ramp2019\2019-05_RAMP_all_page-clicks.zip
D:/Documents/PycharmProjects/LEADING-RAMP/ir_data/Ramp2019\2019-06_RAMP_all_page-clicks.zip
D:/Documents/PycharmProjects/LEADING-RAMP/ir_data/Ramp2019\2019-07_RAMP_all_page-clicks.zip
D:/Documents/PycharmProjects/LEADING-RAMP/ir_data/Ramp2019\2019-08_RAMP_all_page-clicks.zip
D:/Documents/PycharmProjects/LEADING-RAMP/ir_data/Ramp2019\2019-09_RAMP_all_page-clicks.zip
D:/Documents/PycharmProjects/LEADING-RAMP/ir_data/Ramp2019\2019-10_RAMP_all_page-clicks.zip
D:/Documents/PycharmProjects/LEADING-RAMP/ir_data/Ramp2019\2019-11_RAMP_all_page

In [32]:
ut_pc_2019.to_csv("D://Documents//PycharmProjects//LEADING-RAMP//ir_data//Ramp2019//Export/ut_pc_2019.csv", header = True)

In [33]:
unlv_pc_2019 = process_repo("university_nevada_las_vegas")

D:/Documents/PycharmProjects/LEADING-RAMP/ir_data/Ramp2019\2019-01_RAMP_all_page-clicks.zip
D:/Documents/PycharmProjects/LEADING-RAMP/ir_data/Ramp2019\2019-02_RAMP_all_page-clicks.zip
D:/Documents/PycharmProjects/LEADING-RAMP/ir_data/Ramp2019\2019-03_RAMP_all_page-clicks.zip
D:/Documents/PycharmProjects/LEADING-RAMP/ir_data/Ramp2019\2019-04_RAMP_all_page-clicks.zip
D:/Documents/PycharmProjects/LEADING-RAMP/ir_data/Ramp2019\2019-05_RAMP_all_page-clicks.zip
D:/Documents/PycharmProjects/LEADING-RAMP/ir_data/Ramp2019\2019-06_RAMP_all_page-clicks.zip
D:/Documents/PycharmProjects/LEADING-RAMP/ir_data/Ramp2019\2019-07_RAMP_all_page-clicks.zip
D:/Documents/PycharmProjects/LEADING-RAMP/ir_data/Ramp2019\2019-08_RAMP_all_page-clicks.zip
D:/Documents/PycharmProjects/LEADING-RAMP/ir_data/Ramp2019\2019-09_RAMP_all_page-clicks.zip
D:/Documents/PycharmProjects/LEADING-RAMP/ir_data/Ramp2019\2019-10_RAMP_all_page-clicks.zip
D:/Documents/PycharmProjects/LEADING-RAMP/ir_data/Ramp2019\2019-11_RAMP_all_page

In [34]:
unlv_pc_2019.to_csv("D://Documents//PycharmProjects//LEADING-RAMP//ir_data//Ramp2019//Export/unlv_pc_2019.csv", header = True)

In [35]:
unl_pc_2019 = process_repo("university_nebraska_lincoln")

D:/Documents/PycharmProjects/LEADING-RAMP/ir_data/Ramp2019\2019-01_RAMP_all_page-clicks.zip
D:/Documents/PycharmProjects/LEADING-RAMP/ir_data/Ramp2019\2019-02_RAMP_all_page-clicks.zip
D:/Documents/PycharmProjects/LEADING-RAMP/ir_data/Ramp2019\2019-03_RAMP_all_page-clicks.zip
D:/Documents/PycharmProjects/LEADING-RAMP/ir_data/Ramp2019\2019-04_RAMP_all_page-clicks.zip
D:/Documents/PycharmProjects/LEADING-RAMP/ir_data/Ramp2019\2019-05_RAMP_all_page-clicks.zip
D:/Documents/PycharmProjects/LEADING-RAMP/ir_data/Ramp2019\2019-06_RAMP_all_page-clicks.zip
D:/Documents/PycharmProjects/LEADING-RAMP/ir_data/Ramp2019\2019-07_RAMP_all_page-clicks.zip
D:/Documents/PycharmProjects/LEADING-RAMP/ir_data/Ramp2019\2019-08_RAMP_all_page-clicks.zip
D:/Documents/PycharmProjects/LEADING-RAMP/ir_data/Ramp2019\2019-09_RAMP_all_page-clicks.zip
D:/Documents/PycharmProjects/LEADING-RAMP/ir_data/Ramp2019\2019-10_RAMP_all_page-clicks.zip
D:/Documents/PycharmProjects/LEADING-RAMP/ir_data/Ramp2019\2019-11_RAMP_all_page

In [36]:
unl_pc_2019.to_csv("D://Documents//PycharmProjects//LEADING-RAMP//ir_data//Ramp2019//Export/unl_pc_2019.csv", header = True)

In [37]:
uken_pc_2019 = process_repo("university_kentucky")

D:/Documents/PycharmProjects/LEADING-RAMP/ir_data/Ramp2019\2019-01_RAMP_all_page-clicks.zip
D:/Documents/PycharmProjects/LEADING-RAMP/ir_data/Ramp2019\2019-02_RAMP_all_page-clicks.zip
D:/Documents/PycharmProjects/LEADING-RAMP/ir_data/Ramp2019\2019-03_RAMP_all_page-clicks.zip
D:/Documents/PycharmProjects/LEADING-RAMP/ir_data/Ramp2019\2019-04_RAMP_all_page-clicks.zip
D:/Documents/PycharmProjects/LEADING-RAMP/ir_data/Ramp2019\2019-05_RAMP_all_page-clicks.zip
D:/Documents/PycharmProjects/LEADING-RAMP/ir_data/Ramp2019\2019-06_RAMP_all_page-clicks.zip
D:/Documents/PycharmProjects/LEADING-RAMP/ir_data/Ramp2019\2019-07_RAMP_all_page-clicks.zip
D:/Documents/PycharmProjects/LEADING-RAMP/ir_data/Ramp2019\2019-08_RAMP_all_page-clicks.zip
D:/Documents/PycharmProjects/LEADING-RAMP/ir_data/Ramp2019\2019-09_RAMP_all_page-clicks.zip
D:/Documents/PycharmProjects/LEADING-RAMP/ir_data/Ramp2019\2019-10_RAMP_all_page-clicks.zip
D:/Documents/PycharmProjects/LEADING-RAMP/ir_data/Ramp2019\2019-11_RAMP_all_page

In [38]:
uken_pc_2019.to_csv("D://Documents//PycharmProjects//LEADING-RAMP//ir_data//Ramp2019//Export/uken_pc_2019.csv", header = True)

In [39]:
uariz_pc_2019 = process_repo("university_arizona")

D:/Documents/PycharmProjects/LEADING-RAMP/ir_data/Ramp2019\2019-01_RAMP_all_page-clicks.zip
D:/Documents/PycharmProjects/LEADING-RAMP/ir_data/Ramp2019\2019-02_RAMP_all_page-clicks.zip
D:/Documents/PycharmProjects/LEADING-RAMP/ir_data/Ramp2019\2019-03_RAMP_all_page-clicks.zip
D:/Documents/PycharmProjects/LEADING-RAMP/ir_data/Ramp2019\2019-04_RAMP_all_page-clicks.zip
D:/Documents/PycharmProjects/LEADING-RAMP/ir_data/Ramp2019\2019-05_RAMP_all_page-clicks.zip
D:/Documents/PycharmProjects/LEADING-RAMP/ir_data/Ramp2019\2019-06_RAMP_all_page-clicks.zip
D:/Documents/PycharmProjects/LEADING-RAMP/ir_data/Ramp2019\2019-07_RAMP_all_page-clicks.zip
D:/Documents/PycharmProjects/LEADING-RAMP/ir_data/Ramp2019\2019-08_RAMP_all_page-clicks.zip
D:/Documents/PycharmProjects/LEADING-RAMP/ir_data/Ramp2019\2019-09_RAMP_all_page-clicks.zip
D:/Documents/PycharmProjects/LEADING-RAMP/ir_data/Ramp2019\2019-10_RAMP_all_page-clicks.zip
D:/Documents/PycharmProjects/LEADING-RAMP/ir_data/Ramp2019\2019-11_RAMP_all_page

In [40]:
uariz_pc_2019.to_csv("D://Documents//PycharmProjects//LEADING-RAMP//ir_data//Ramp2019//Export/uariz_pc_2019.csv", header = True)

In [41]:
swarthmore_pc_2019 = process_repo("swarthmore_college")

D:/Documents/PycharmProjects/LEADING-RAMP/ir_data/Ramp2019\2019-01_RAMP_all_page-clicks.zip
D:/Documents/PycharmProjects/LEADING-RAMP/ir_data/Ramp2019\2019-02_RAMP_all_page-clicks.zip
D:/Documents/PycharmProjects/LEADING-RAMP/ir_data/Ramp2019\2019-03_RAMP_all_page-clicks.zip
D:/Documents/PycharmProjects/LEADING-RAMP/ir_data/Ramp2019\2019-04_RAMP_all_page-clicks.zip
D:/Documents/PycharmProjects/LEADING-RAMP/ir_data/Ramp2019\2019-05_RAMP_all_page-clicks.zip
D:/Documents/PycharmProjects/LEADING-RAMP/ir_data/Ramp2019\2019-06_RAMP_all_page-clicks.zip
D:/Documents/PycharmProjects/LEADING-RAMP/ir_data/Ramp2019\2019-07_RAMP_all_page-clicks.zip
D:/Documents/PycharmProjects/LEADING-RAMP/ir_data/Ramp2019\2019-08_RAMP_all_page-clicks.zip
D:/Documents/PycharmProjects/LEADING-RAMP/ir_data/Ramp2019\2019-09_RAMP_all_page-clicks.zip
D:/Documents/PycharmProjects/LEADING-RAMP/ir_data/Ramp2019\2019-10_RAMP_all_page-clicks.zip
D:/Documents/PycharmProjects/LEADING-RAMP/ir_data/Ramp2019\2019-11_RAMP_all_page

In [42]:
swarthmore_pc_2019.to_csv("D://Documents//PycharmProjects//LEADING-RAMP//ir_data//Ramp2019//Export/swarthmore_pc_2019.csv", header = True)

In [43]:
samhouston_pc_2019 = process_repo("sam_houston_state_university")

D:/Documents/PycharmProjects/LEADING-RAMP/ir_data/Ramp2019\2019-01_RAMP_all_page-clicks.zip
D:/Documents/PycharmProjects/LEADING-RAMP/ir_data/Ramp2019\2019-02_RAMP_all_page-clicks.zip
D:/Documents/PycharmProjects/LEADING-RAMP/ir_data/Ramp2019\2019-03_RAMP_all_page-clicks.zip
D:/Documents/PycharmProjects/LEADING-RAMP/ir_data/Ramp2019\2019-04_RAMP_all_page-clicks.zip
D:/Documents/PycharmProjects/LEADING-RAMP/ir_data/Ramp2019\2019-05_RAMP_all_page-clicks.zip
D:/Documents/PycharmProjects/LEADING-RAMP/ir_data/Ramp2019\2019-06_RAMP_all_page-clicks.zip
D:/Documents/PycharmProjects/LEADING-RAMP/ir_data/Ramp2019\2019-07_RAMP_all_page-clicks.zip
D:/Documents/PycharmProjects/LEADING-RAMP/ir_data/Ramp2019\2019-08_RAMP_all_page-clicks.zip
D:/Documents/PycharmProjects/LEADING-RAMP/ir_data/Ramp2019\2019-09_RAMP_all_page-clicks.zip
D:/Documents/PycharmProjects/LEADING-RAMP/ir_data/Ramp2019\2019-10_RAMP_all_page-clicks.zip
D:/Documents/PycharmProjects/LEADING-RAMP/ir_data/Ramp2019\2019-11_RAMP_all_page

In [44]:
samhouston_pc_2019.to_csv("D://Documents//PycharmProjects//LEADING-RAMP//ir_data//Ramp2019//Export/samhouston_pc_2019.csv", header = True)

In [45]:
rutgers_pc_2019 = process_repo("rutgers_university")

D:/Documents/PycharmProjects/LEADING-RAMP/ir_data/Ramp2019\2019-01_RAMP_all_page-clicks.zip
D:/Documents/PycharmProjects/LEADING-RAMP/ir_data/Ramp2019\2019-02_RAMP_all_page-clicks.zip
D:/Documents/PycharmProjects/LEADING-RAMP/ir_data/Ramp2019\2019-03_RAMP_all_page-clicks.zip
D:/Documents/PycharmProjects/LEADING-RAMP/ir_data/Ramp2019\2019-04_RAMP_all_page-clicks.zip
D:/Documents/PycharmProjects/LEADING-RAMP/ir_data/Ramp2019\2019-05_RAMP_all_page-clicks.zip
D:/Documents/PycharmProjects/LEADING-RAMP/ir_data/Ramp2019\2019-06_RAMP_all_page-clicks.zip
D:/Documents/PycharmProjects/LEADING-RAMP/ir_data/Ramp2019\2019-07_RAMP_all_page-clicks.zip
D:/Documents/PycharmProjects/LEADING-RAMP/ir_data/Ramp2019\2019-08_RAMP_all_page-clicks.zip
D:/Documents/PycharmProjects/LEADING-RAMP/ir_data/Ramp2019\2019-09_RAMP_all_page-clicks.zip
D:/Documents/PycharmProjects/LEADING-RAMP/ir_data/Ramp2019\2019-10_RAMP_all_page-clicks.zip
D:/Documents/PycharmProjects/LEADING-RAMP/ir_data/Ramp2019\2019-11_RAMP_all_page

In [46]:
rutgers_pc_2019.to_csv("D://Documents//PycharmProjects//LEADING-RAMP//ir_data//Ramp2019//Export/rutgers_pc_2019.csv", header = True)

In [47]:
northeast_pc_2019 = process_repo("northeastern_university")

D:/Documents/PycharmProjects/LEADING-RAMP/ir_data/Ramp2019\2019-01_RAMP_all_page-clicks.zip
D:/Documents/PycharmProjects/LEADING-RAMP/ir_data/Ramp2019\2019-02_RAMP_all_page-clicks.zip
D:/Documents/PycharmProjects/LEADING-RAMP/ir_data/Ramp2019\2019-03_RAMP_all_page-clicks.zip
D:/Documents/PycharmProjects/LEADING-RAMP/ir_data/Ramp2019\2019-04_RAMP_all_page-clicks.zip
D:/Documents/PycharmProjects/LEADING-RAMP/ir_data/Ramp2019\2019-05_RAMP_all_page-clicks.zip
D:/Documents/PycharmProjects/LEADING-RAMP/ir_data/Ramp2019\2019-06_RAMP_all_page-clicks.zip
D:/Documents/PycharmProjects/LEADING-RAMP/ir_data/Ramp2019\2019-07_RAMP_all_page-clicks.zip
D:/Documents/PycharmProjects/LEADING-RAMP/ir_data/Ramp2019\2019-08_RAMP_all_page-clicks.zip
D:/Documents/PycharmProjects/LEADING-RAMP/ir_data/Ramp2019\2019-09_RAMP_all_page-clicks.zip
D:/Documents/PycharmProjects/LEADING-RAMP/ir_data/Ramp2019\2019-10_RAMP_all_page-clicks.zip
D:/Documents/PycharmProjects/LEADING-RAMP/ir_data/Ramp2019\2019-11_RAMP_all_page

In [48]:
northeast_pc_2019.to_csv("D://Documents//PycharmProjects//LEADING-RAMP//ir_data//Ramp2019//Export/northeast_pc_2019.csv", header = True)

In [49]:
unm_pc_2019 = process_repo("university_new_mexico")

D:/Documents/PycharmProjects/LEADING-RAMP/ir_data/Ramp2019\2019-01_RAMP_all_page-clicks.zip
D:/Documents/PycharmProjects/LEADING-RAMP/ir_data/Ramp2019\2019-02_RAMP_all_page-clicks.zip
D:/Documents/PycharmProjects/LEADING-RAMP/ir_data/Ramp2019\2019-03_RAMP_all_page-clicks.zip
D:/Documents/PycharmProjects/LEADING-RAMP/ir_data/Ramp2019\2019-04_RAMP_all_page-clicks.zip
D:/Documents/PycharmProjects/LEADING-RAMP/ir_data/Ramp2019\2019-05_RAMP_all_page-clicks.zip
D:/Documents/PycharmProjects/LEADING-RAMP/ir_data/Ramp2019\2019-06_RAMP_all_page-clicks.zip
D:/Documents/PycharmProjects/LEADING-RAMP/ir_data/Ramp2019\2019-07_RAMP_all_page-clicks.zip
D:/Documents/PycharmProjects/LEADING-RAMP/ir_data/Ramp2019\2019-08_RAMP_all_page-clicks.zip
D:/Documents/PycharmProjects/LEADING-RAMP/ir_data/Ramp2019\2019-09_RAMP_all_page-clicks.zip
D:/Documents/PycharmProjects/LEADING-RAMP/ir_data/Ramp2019\2019-10_RAMP_all_page-clicks.zip
D:/Documents/PycharmProjects/LEADING-RAMP/ir_data/Ramp2019\2019-11_RAMP_all_page

In [50]:
unm_pc_2019.to_csv("D://Documents//PycharmProjects//LEADING-RAMP//ir_data//Ramp2019//Export/unm_pc_2019.csv", header = True)

In [51]:
nky_pc_2019 = process_repo("northern_kentucky_university")

D:/Documents/PycharmProjects/LEADING-RAMP/ir_data/Ramp2019\2019-01_RAMP_all_page-clicks.zip
D:/Documents/PycharmProjects/LEADING-RAMP/ir_data/Ramp2019\2019-02_RAMP_all_page-clicks.zip
D:/Documents/PycharmProjects/LEADING-RAMP/ir_data/Ramp2019\2019-03_RAMP_all_page-clicks.zip
D:/Documents/PycharmProjects/LEADING-RAMP/ir_data/Ramp2019\2019-04_RAMP_all_page-clicks.zip
D:/Documents/PycharmProjects/LEADING-RAMP/ir_data/Ramp2019\2019-05_RAMP_all_page-clicks.zip
D:/Documents/PycharmProjects/LEADING-RAMP/ir_data/Ramp2019\2019-06_RAMP_all_page-clicks.zip
D:/Documents/PycharmProjects/LEADING-RAMP/ir_data/Ramp2019\2019-07_RAMP_all_page-clicks.zip
D:/Documents/PycharmProjects/LEADING-RAMP/ir_data/Ramp2019\2019-08_RAMP_all_page-clicks.zip
D:/Documents/PycharmProjects/LEADING-RAMP/ir_data/Ramp2019\2019-09_RAMP_all_page-clicks.zip
D:/Documents/PycharmProjects/LEADING-RAMP/ir_data/Ramp2019\2019-10_RAMP_all_page-clicks.zip
D:/Documents/PycharmProjects/LEADING-RAMP/ir_data/Ramp2019\2019-11_RAMP_all_page

In [52]:
nky_pc_2019.to_csv("D://Documents//PycharmProjects//LEADING-RAMP//ir_data//Ramp2019//Export/nky_pc_2019.csv", header = True)

In [53]:
umich_pc_2019 = process_repo("university_michigan")

D:/Documents/PycharmProjects/LEADING-RAMP/ir_data/Ramp2019\2019-01_RAMP_all_page-clicks.zip
D:/Documents/PycharmProjects/LEADING-RAMP/ir_data/Ramp2019\2019-02_RAMP_all_page-clicks.zip
D:/Documents/PycharmProjects/LEADING-RAMP/ir_data/Ramp2019\2019-03_RAMP_all_page-clicks.zip
D:/Documents/PycharmProjects/LEADING-RAMP/ir_data/Ramp2019\2019-04_RAMP_all_page-clicks.zip
D:/Documents/PycharmProjects/LEADING-RAMP/ir_data/Ramp2019\2019-05_RAMP_all_page-clicks.zip
D:/Documents/PycharmProjects/LEADING-RAMP/ir_data/Ramp2019\2019-06_RAMP_all_page-clicks.zip
D:/Documents/PycharmProjects/LEADING-RAMP/ir_data/Ramp2019\2019-07_RAMP_all_page-clicks.zip
D:/Documents/PycharmProjects/LEADING-RAMP/ir_data/Ramp2019\2019-08_RAMP_all_page-clicks.zip
D:/Documents/PycharmProjects/LEADING-RAMP/ir_data/Ramp2019\2019-09_RAMP_all_page-clicks.zip
D:/Documents/PycharmProjects/LEADING-RAMP/ir_data/Ramp2019\2019-10_RAMP_all_page-clicks.zip
D:/Documents/PycharmProjects/LEADING-RAMP/ir_data/Ramp2019\2019-11_RAMP_all_page

In [54]:
umich_pc_2019.to_csv("D://Documents//PycharmProjects//LEADING-RAMP//ir_data//Ramp2019//Export/umich_pc_2019.csv", header = True)

In [55]:
umaryland_pc_2019 = process_repo("maryland_drum")

D:/Documents/PycharmProjects/LEADING-RAMP/ir_data/Ramp2019\2019-01_RAMP_all_page-clicks.zip
D:/Documents/PycharmProjects/LEADING-RAMP/ir_data/Ramp2019\2019-02_RAMP_all_page-clicks.zip
D:/Documents/PycharmProjects/LEADING-RAMP/ir_data/Ramp2019\2019-03_RAMP_all_page-clicks.zip
D:/Documents/PycharmProjects/LEADING-RAMP/ir_data/Ramp2019\2019-04_RAMP_all_page-clicks.zip
D:/Documents/PycharmProjects/LEADING-RAMP/ir_data/Ramp2019\2019-05_RAMP_all_page-clicks.zip
D:/Documents/PycharmProjects/LEADING-RAMP/ir_data/Ramp2019\2019-06_RAMP_all_page-clicks.zip
D:/Documents/PycharmProjects/LEADING-RAMP/ir_data/Ramp2019\2019-07_RAMP_all_page-clicks.zip
D:/Documents/PycharmProjects/LEADING-RAMP/ir_data/Ramp2019\2019-08_RAMP_all_page-clicks.zip
D:/Documents/PycharmProjects/LEADING-RAMP/ir_data/Ramp2019\2019-09_RAMP_all_page-clicks.zip
D:/Documents/PycharmProjects/LEADING-RAMP/ir_data/Ramp2019\2019-10_RAMP_all_page-clicks.zip
D:/Documents/PycharmProjects/LEADING-RAMP/ir_data/Ramp2019\2019-11_RAMP_all_page

In [56]:
umaryland_pc_2019.to_csv("D://Documents//PycharmProjects//LEADING-RAMP//ir_data//Ramp2019//Export/umaryland_pc_2019.csv", header = True)

In [57]:
kansas_pc_2019 = process_repo("kansas_state_university")

D:/Documents/PycharmProjects/LEADING-RAMP/ir_data/Ramp2019\2019-01_RAMP_all_page-clicks.zip
D:/Documents/PycharmProjects/LEADING-RAMP/ir_data/Ramp2019\2019-02_RAMP_all_page-clicks.zip
D:/Documents/PycharmProjects/LEADING-RAMP/ir_data/Ramp2019\2019-03_RAMP_all_page-clicks.zip
D:/Documents/PycharmProjects/LEADING-RAMP/ir_data/Ramp2019\2019-04_RAMP_all_page-clicks.zip
D:/Documents/PycharmProjects/LEADING-RAMP/ir_data/Ramp2019\2019-05_RAMP_all_page-clicks.zip
D:/Documents/PycharmProjects/LEADING-RAMP/ir_data/Ramp2019\2019-06_RAMP_all_page-clicks.zip
D:/Documents/PycharmProjects/LEADING-RAMP/ir_data/Ramp2019\2019-07_RAMP_all_page-clicks.zip
D:/Documents/PycharmProjects/LEADING-RAMP/ir_data/Ramp2019\2019-08_RAMP_all_page-clicks.zip
D:/Documents/PycharmProjects/LEADING-RAMP/ir_data/Ramp2019\2019-09_RAMP_all_page-clicks.zip
D:/Documents/PycharmProjects/LEADING-RAMP/ir_data/Ramp2019\2019-10_RAMP_all_page-clicks.zip
D:/Documents/PycharmProjects/LEADING-RAMP/ir_data/Ramp2019\2019-11_RAMP_all_page

In [58]:
kansas_pc_2019.to_csv("D://Documents//PycharmProjects//LEADING-RAMP//ir_data//Ramp2019//Export/kansas_pc_2019.csv", header = True)

In [59]:
umontana_pc_2019 = process_repo("university_montana")

D:/Documents/PycharmProjects/LEADING-RAMP/ir_data/Ramp2019\2019-01_RAMP_all_page-clicks.zip
D:/Documents/PycharmProjects/LEADING-RAMP/ir_data/Ramp2019\2019-02_RAMP_all_page-clicks.zip
D:/Documents/PycharmProjects/LEADING-RAMP/ir_data/Ramp2019\2019-03_RAMP_all_page-clicks.zip
D:/Documents/PycharmProjects/LEADING-RAMP/ir_data/Ramp2019\2019-04_RAMP_all_page-clicks.zip
D:/Documents/PycharmProjects/LEADING-RAMP/ir_data/Ramp2019\2019-05_RAMP_all_page-clicks.zip
D:/Documents/PycharmProjects/LEADING-RAMP/ir_data/Ramp2019\2019-06_RAMP_all_page-clicks.zip
D:/Documents/PycharmProjects/LEADING-RAMP/ir_data/Ramp2019\2019-07_RAMP_all_page-clicks.zip
D:/Documents/PycharmProjects/LEADING-RAMP/ir_data/Ramp2019\2019-08_RAMP_all_page-clicks.zip
D:/Documents/PycharmProjects/LEADING-RAMP/ir_data/Ramp2019\2019-09_RAMP_all_page-clicks.zip
D:/Documents/PycharmProjects/LEADING-RAMP/ir_data/Ramp2019\2019-10_RAMP_all_page-clicks.zip
D:/Documents/PycharmProjects/LEADING-RAMP/ir_data/Ramp2019\2019-11_RAMP_all_page

In [60]:
umontana_pc_2019.to_csv("D://Documents//PycharmProjects//LEADING-RAMP//ir_data//Ramp2019//Export/umontana_pc_2019.csv", header = True)